In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [9]:
import warnings

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        # total of the stats for basic and advanced (last row)
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        # highest mark of each of the columns for basic and advanced (all rows except last row)
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
            
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 11126
200 / 11126
300 / 11126
400 / 11126
500 / 11126
600 / 11126
700 / 11126
800 / 11126
900 / 11126
1000 / 11126
1100 / 11126
1200 / 11126
1300 / 11126
1400 / 11126
1500 / 11126
1600 / 11126
1700 / 11126
1800 / 11126
1900 / 11126
2000 / 11126
2100 / 11126
2200 / 11126
2300 / 11126
2400 / 11126
2500 / 11126
2600 / 11126
2700 / 11126
2800 / 11126
2900 / 11126
3000 / 11126
3100 / 11126
3200 / 11126
3300 / 11126
3400 / 11126
3500 / 11126
3600 / 11126
3700 / 11126
3800 / 11126
3900 / 11126
4000 / 11126
4100 / 11126
4200 / 11126
4300 / 11126
4400 / 11126
4500 / 11126
4600 / 11126
4700 / 11126
4800 / 11126
4900 / 11126
5000 / 11126
5100 / 11126
5200 / 11126
5300 / 11126
5400 / 11126
5500 / 11126
5600 / 11126
5700 / 11126
5800 / 11126
5900 / 11126
6000 / 11126
6100 / 11126
6200 / 11126
6300 / 11126
6400 / 11126
6500 / 11126
6600 / 11126
6700 / 11126
6800 / 11126
6900 / 11126
7000 / 11126
7100 / 11126
7200 / 11126
7300 / 11126
7400 / 11126
7500 / 11126
7600 / 11126
7700 / 11126
7800 / 1

In [10]:
games_df = pd.concat(games, ignore_index=True)

In [11]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22247,240.0,240.0,40.0,87.0,0.460,10.0,29.0,0.345,12.0,16.0,...,33.3,32.5,189.0,122.0,BOS,105,0,2024,2024-05-27,False
22248,240.0,240.0,39.0,74.0,0.527,11.0,24.0,0.458,16.0,25.0,...,100.0,34.6,208.0,120.0,DAL,100,1,2024,2024-05-28,True
22249,240.0,240.0,34.0,81.0,0.420,14.0,40.0,0.350,18.0,21.0,...,21.8,41.4,154.0,115.0,MIN,105,0,2024,2024-05-28,False
22250,240.0,240.0,46.0,84.0,0.548,15.0,34.0,0.441,17.0,23.0,...,22.3,43.5,223.0,139.0,MIN,103,1,2024,2024-05-30,True


In [12]:
games_df.to_csv("nba_games.csv")